# Downloading meteorological data
In `nlmod` we can download meteorological data from the Dutch meteorlogical institute, using the Python package `hydropandas` in the background. In the following sections we will show different options for downloading and storing meteorological data:
- Calculate recharge by subtracting evaporation from precipitation
- Store precipitation and evaporation to seperate variables
- Seperate the meteorological measurement values and the station locations

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import nlmod

In [ ]:
nlmod.util.get_color_logger("INFO")
nlmod.show_versions()

## Generate a model Datset
First we create a model Dataset for a specific part of the Netherlands.

In [ ]:
extent = [100_000, 110_000, 400_000, 410_000]
ds = nlmod.get_ds(extent)
nlmod.plot.get_map(extent, background=True);

We need to set the time dimension of the model dataset, so we know what period we need to download meteorological data for.

We set our simulation time to be every day of the year 2024, and we set the start of our siumlation at the start of 2023. In this way we start with a steady state stress period with average meteorological data from the year 2023.

In [ ]:
time = pd.date_range("2024-01-01", "2025-1-1", freq="D")
start = "2023-01-01"
ds = nlmod.time.set_ds_time(ds, start=start, time=time)
# show the contents of our time dimension
ds.time

## Calculate recharge by subtracting evaporation from precipitation
We use the method `nlmod.read.knmi.get_recharge` to download recharge data from the KNMI for our entire model area. This method downloads precipitation-data ("RD") from precipitation-stations ("neerslagstations") and Makking-evaporation data ("EV24") from the meteoroogical stations (weerstations) of the KNMI. As can be seen from the info-messages below, we see that data from several precipitation-stations is downloaded, and date from one meteoroogical station.

By default, `nlmod.read.knmi.get_recharge` subtracts the evaporation form the precipitation, to calculate the groundwater recharge. This recharge is returned as the variable `recharge` in the returned xarray Dataset.

In [ ]:
meteo = nlmod.read.knmi.get_recharge(ds)
# show the contents of the returned dataset
meteo

We can plot the average rainfall over the entire time period. We see a Thiessen-polygon containing the precipitation-values.

In [ ]:
nlmod.plot.map_array(meteo['recharge'][1:].mean(dim='time'), cmap='Blues', ds=ds);

## Store precipitation and evaporation to seperate variables
We can also return the precipitation and evaporation data in separate variables `recharge` and `evaporation`, to be used in the RCH-, EVT- and UZF-packages. We can use `ds.update(meteo)` to add these variables to the model dataset `ds`.

In [ ]:
meteo = nlmod.read.knmi.get_recharge(ds, method="separate")
# show the contents of the returned dataset
meteo

In [ ]:
f, ax = nlmod.plot.get_map(extent, ncols=2)
qm = nlmod.plot.data_array(meteo['recharge'][1:].mean(dim='time') * 1000, cmap='Blues', ds=ds, ax=ax[0])
nlmod.plot.colorbar_inside(qm, ax=ax[0], label='Recharge (mm/day)')
qm = nlmod.plot.data_array(meteo['evaporation'][1:].mean(dim='time') * 1000, cmap='Blues', ds=ds, ax=ax[1])
nlmod.plot.colorbar_inside(qm, ax=ax[1], label='Evaporation (mm/day)');

## Seperate the meteorological measurement values and the station locations
The previous data is transformed to xarray DataArrays with dimensions `time`, `y` and `x`, with many duplicate values for cells near to the same station. If we do not want this 3d-array with a lot of duplicate values, we can use the optional parameter `add_stn_dimensions`. If we set this parameter to True, the `recharge` and `evaporation` variables contain the dimensions `stn_RD` and `stn_EV24`, and the variables `recharge_stn` and `evaporation_stn` contain the station-number that is used in every model cell. This way of storing meteorlogiscal data in the model dataset is also supported in the generation of the RCH- EVT- and UZF-packages.

In [ ]:
meteo = nlmod.read.knmi.get_recharge(ds, add_stn_dimensions=True)
# show the contents of the returned dataset
meteo

In [ ]:
f, ax = nlmod.plot.get_map(extent, ncols=2)
qm = nlmod.plot.data_array(meteo['recharge_stn'], ds=ds, ax=ax[0])
nlmod.plot.colorbar_inside(qm, ax=ax[0], label='Recharge station')
qm = nlmod.plot.data_array(meteo['evaporation_stn'], ds=ds, ax=ax[1])
nlmod.plot.colorbar_inside(qm, ax=ax[1], label='Evaporation station');

We can now better plot the measurement-values from the indiviudeal measurement-stations.

In [ ]:
f, axes = plt.subplots(figsize=(10, 10), nrows=2, sharex=True, layout="constrained")

meteo['recharge'].to_pandas().plot(ax=axes[0])
axes[0].set_ylabel('Recharge (m/day)')
meteo['evaporation'].to_pandas().plot(ax=axes[1])
axes[1].set_ylabel('Evaporation (m/day)');